In [2]:
import functools
import re
from dataclasses import dataclass
from typing import Dict, List, Optional

from conda.models import version as cv

In [3]:
with open("conflicts.txt","r") as f:
    conflictstr = f.read()

In [4]:
@dataclass
class Constraint:
    dependency: Optional[
        str
    ]  # Name of dependency that requires the conflicting package
    versions: cv.VersionSpec  # List of versions for the conflicting package


@dataclass
class ConstraintGroup:
    name: str  # Name of conflicting package
    constraints: List[Constraint]

    def merge(self) -> cv.VersionSpec:
        return functools.reduce(
            lambda v1, v2: v1.merge(v2), (c.versions for c in self.constraints)
        )


def parse_conflicts(conflicts: str) -> List[ConstraintGroup]:
    # groups separated by blank line
    return [_parse_group(group) for group in conflicts.split("\n\n")]


def _parse_group(group: str) -> ConstraintGroup:
    groupname = re.compile(r"Package (\S+) conflicts for:")
    lines = group.split("\n")
    # first line gives name
    match = groupname.match(lines[0])
    name = match and match.group(1)
    # look for versions in subsequent lines
    return ConstraintGroup(
        name, [c for c in (_parse_line(name, line) for line in lines[1:] if line) if c]
    )


def _parse_line(name: str, line: str) -> Constraint:
    versionre = re.compile(r"(?:(\S+) .*)?" + name + r"(?:\[version='([^']+)'[],])?")
    match = versionre.match(line)
    if match:
        dep = match.group(1)
        spec = match.group(2)
        if spec is None:
            return None
        versions = cv.VersionSpec(spec)
        return Constraint(dep, versions)
    else:
        logging.warning(f"Unable to parse {name} version in: {line!r}")
        return None


conflicts = parse_conflicts(conflictstr)
conflicts

[ConstraintGroup(name=None, constraints=[]),
 ConstraintGroup(name=None, constraints=[]),
 ConstraintGroup(name='libzlib', constraints=[Constraint(dependency='matplotlib', versions=VersionSpec('1.2.11|1.2.11|>=1.2.11,<1.3.0a0')), Constraint(dependency='pytrends', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='dataclasses', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='requests', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='numpy', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='pip', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='mplfinance', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='pandas', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='ipython', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='python-dateutil', versions=VersionSpec('>=1.2.11,<1.3.0a0')), Constraint(dependency='ipykernel', versions=VersionSpec('>=1.2.11,<

In [5]:
conflicts[0].merge()

TypeError: reduce() of empty sequence with no initial value

In [ ]:
conflicts[0].merge().match("6.1")

True

In [ ]:
conflicts[1].merge()

VersionSpec('2.7.*|3.5.*|3.6.*|>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.8,<3.9.0a0|>=3.5,<3.6.0a0|3.4.*,2.7.*|3.5.*|3.6.*|>=2.7,<2.8.0a0|>=3.6,<3.7.0a0|>=3.8,<3.9.0a0|>=3.7,<3.8.0a0|>=3.5,<3.6.0a0|3.4.*|>3|>=3.5|<3.0.0|>=3.6,>=3.4,3.7.*|3.8.*')

In [ ]:
conflicts[1].merge().match(3.8)

True

In [ ]:
conflicts[2]

ConstraintGroup(name='ca-certificates', constraints=[])

In [ ]:
conflicts[3]

ConstraintGroup(name='setuptools', constraints=[Constraint(dependency='instrain', versions=VersionSpec('>=40.0'))])

In [ ]:
conflicts[3].merge().match("40.0")

True

In [ ]:
conflicts[4]

ConstraintGroup(name='libgcc-ng', constraints=[Constraint(dependency='samtools', versions=VersionSpec('>=7.2.0')), Constraint(dependency='samtools', versions=VersionSpec('>=4.9|>=7.3.0'))])

In [ ]:
conflicts[4].merge().match("7.3.0")

True

In [ ]:
conflicts[5]

ConstraintGroup(name='pypy3.6', constraints=[Constraint(dependency='instrain', versions=VersionSpec('7.3.0.*|7.3.1.*|>=7.3.1')), Constraint(dependency='awscli', versions=VersionSpec('7.3.*|7.3.0.*|7.3.1.*'))])

In [ ]:
conflicts[5].merge().match("7.3.1")

True

In [ ]:
conflicts[6]

ConstraintGroup(name='bzip2', constraints=[Constraint(dependency='samtools', versions=VersionSpec('1.0.*|>=1.0.6,<2.0a0|>=1.0.8,<2.0a0')), Constraint(dependency='instrain', versions=VersionSpec('>=1.0.6,<2.0a0|>=1.0.8,<2.0a0')), Constraint(dependency='awscli', versions=VersionSpec('>=1.0.6,<2.0a0|>=1.0.8,<2.0a0'))])

In [ ]:
conflicts[6].merge().match("1.0.8")

True

In [ ]:
conflicts[7]

ConstraintGroup(name='zlib', constraints=[Constraint(dependency='samtools', versions=VersionSpec('1.2.11.*|>=1.2.11,<1.3.0a0|1.2.8.*|1.2.8')), Constraint(dependency='samtools', versions=VersionSpec('1.2.*|1.2.11'))])

In [ ]:
conflicts[7].merge().match("1.2.11")

True

In [ ]:
conflicts[8]

ConstraintGroup(name='samtools', constraints=[Constraint(dependency='instrain', versions=VersionSpec('1.3|1.3.1.*|1.3.1|1.5.*|1.6.*|1.7|1.7.*|1.9.*|>=1.4.1|>=1.4.1,<1.5|>=1.4,<1.5|>=1.3,<1.4|>=1.3'))])

In [ ]:
conflicts[8].merge().match("1.3")

True

In [ ]:
conflicts[9]

ConstraintGroup(name='openssl', constraints=[Constraint(dependency='samtools', versions=VersionSpec('1.0.*|>=1.0.2o,<1.0.3a|>=1.0.2m,<1.0.3a')), Constraint(dependency='samtools', versions=VersionSpec('>=1.0.2p,<1.0.3a|>=1.0.2r,<1.0.3a|>=1.1.1a,<1.1.2a'))])

In [ ]:
conflicts[9].merge().match("1.1.1a")

True

In [ ]:
conflicts[10]

ConstraintGroup(name='_libgcc_mutex', constraints=[Constraint(dependency='samtools', versions=VersionSpec('*|0.1')), Constraint(dependency='python=3.8', versions=VersionSpec('*|0.1'))])

In [ ]:
conflicts[10].merge().match("0.1")

True

In [ ]:
len(conflicts)

11